# THE GREAT FILE DE-DUPLICATOR
### A solution to my fractured backups
A program of 1000 lines begins with dependency imports


In [1]:
from pathlib import Path
from multiprocess import Pool, cpu_count
from pprint import pprint, pformat
import pandas as pd
from IPython.display import clear_output, display, HTML
import sys
import time
from dataclasses import dataclass
#from functools import partial


print('done')

done


### Enter your parameters
Enter where you want to output your analysis files, what directories you want to analyze for duplicates and which folder names you wish to exclude from the analysis.
Mind the backslashes playing nice with python on windows.

**THE ORDER THE deDupeDirs ARE IN ARE THE PREFERRED ORDER FOR RETENTION IF DRY RUN IS DISABLED! MAKE SURE YOU ARE OK WITH LATER ENTRIES IN THE LIST HAVING DUPLICATE FILES DISABLED!**


In [2]:
dedupeDirs = (r'C:\Users\Ryan\Desktop',
               r'D:\Users\Ryan',
               'G:',
               'F:',
               #'H:\\',
               )
excludeDirs = {'$RECYCLE.BIN',
               'gdpr data',
               'steamapps',
               'Google Dump',
               'Windows',
              }

excludeExtensions = {'.example',    
}

outputDir = Path(r'C:\Users\Ryan\Desktop')

#Assign file retention priority to dedupeDirs based on order
#Early entries in the list will keep their files over later entries with duplicate files
print(dedupeDirs)

('C:\\Users\\Ryan\\Desktop', 'D:\\Users\\Ryan', 'G:', 'F:')


# Index all files in directories
### For a directories listed in "deDupeDirs", get the filenames, last modified dates, and file size. 
End result is a dataframe with all files listed within those dirs.

Start off with functions

In [3]:
#@dataclass
#class FileAttr():
#    name: str
#    path: Path
#    size: int
#    mtime: int
    
def fileIndexer(workDir, excludeDirs, excludeExtensions): #args #*args
    #args, not *args to work with pool
    #workDir, excludeDirs, excludeExtensions = args[0], args[1], args[2]
    #trying this partial approach when calling pool to simulate starmap, but still use imap
    try:
        from FileAttr import FileAttr
        from pathlib import Path
        print(f'fileIndexer start for {workDir}')
        if not isinstance(workDir, Path):
            workDir = Path(workDir)
        fileAttrs = []
        errors    = []
        for file in workDir.glob('**/*'):
            try:
                if (   file.is_file() 
                        and not excludeDirs.intersection(set(file.parts)) 
                        and not excludeExtensions.intersection(set(file.suffixes))
                   ):
                    fa = FileAttr(file.name, file,file.stat().st_size, file.stat().st_mtime)#, set(file.parts))
                    fileAttrs.append(fa)
            except Exception as e:
                    errors.append([file, e])
        print(f'fileIndexer end for {workDir}')
        return fileAttrs, errors
    except Exception as e:
        return e

def myIterator():
    for dedupeDir in dedupeDirs:
        yield  dedupeDir, excludeDirs, excludeExtensions

Index the files with multiprocessing and load to df

In [ ]:
%%time
fileAttrs  = []
errors     = []
#if I needed to use imap here, look at functools.partial to replace the myIterator
with Pool(processes = int(cpu_count()/2)) as p:
    for fileAttribute, error in p.starmap(func = fileIndexer, iterable = myIterator()):
        fileAttrs+= fileAttribute
        errors += error
        
df = pd.DataFrame(fileAttrs)
df

### Save your progress
The above steps could take a while depending on how much you've hoarded, pickle the output to noodle with at a later date

In [ ]:
%%time
df.to_pickle(r'C:\Users\Ryan\Desktop\filelist.pkl')
df.to_csv('filelist.csv', index = False)
print('pickled and csved df')

### Reload your progress
Reload if you saved to a JSON file.  You'll need to run imports and the parameter code blocks before this.

In [ ]:
%%time
df = pd.read_pickle(r'C:\Users\Ryan\Desktop\filelist.pkl')                       
print('loaded from file')    

### Calculate CRC for files

 Define functions used in duplicate processing

In [ ]:
def mpCrc32(fileName):
    #have to import within function because of multiprocessing
    from zlib import crc32
    try:
        with open(fileName, 'rb') as fh:
            hash = 0
            while True:
                s = fh.read(65536)
                if not s:
                    break
                hash = crc32(s, hash)
            return "%08X" % (hash & 0xFFFFFFFF)
    except Exception as e:
        x = f'{e}, {fileName}'
        return x       

print('done')

Calculate CRCs, takes a long time with the high disk IO to read every file

In [ ]:
%%time
#shuffle your dataframe to pull randomly and so we don't linearly iterate over folders and end up doing one drive at at time
df = df.sample(frac = 1)

crcList = []
with Pool(processes = int(cpu_count()/2), maxtasksperchild = 500) as p:
    for cnt, crc in enumerate(p.imap(func = mpCrc32, iterable = df['path'], chunksize = 5000)):
        crcList.append(crc)
        sys.stderr.write(f'\r{cnt/len(df):.0%}')
df['crc'] = crcList
df

## Checkpoint your progress  after calculating CRCs
It takes a while to get this far since it's basically a full disk read, save your progress!

In [ ]:
%%time
df.to_pickle(r'C:\Users\Ryan\Desktop\filelistCRC.pkl')
df.to_csv('filelistCRC.csv', index = False)
print('pickled CRC populated dataframe and csved df')

### Reload progress 

In [101]:
%%time
df = pd.read_pickle(r'C:\Users\Ryan\Desktop\filelistCRC.pkl')                       
print('loaded from file')
print(f'{len(df)=}')
df

loaded from file
len(df)=252607
Wall time: 4.98 s


,name,path,size,mtime,crc
201024,math_ops.cpython-37.pyc,D:\Users\Ryan\Desktop\dfl-venv\Lib\site-packag...,602,1.582502e+09,E1757630
167726,__init__.py,D:\Users\Ryan\AppData\Local\Programs\Python\Py...,0,1.586105e+09,00000000
91944,EC7F06D0C44964357DF15001B64BF99DABFFFAFB,D:\Users\Ryan\AppData\Local\Mozilla\Firefox\Pr...,17738,1.599964e+09,FC1DE324
199764,gen_training_ops.py,D:\Users\Ryan\Desktop\dfl-venv\Lib\site-packag...,24963,1.582502e+09,71AED58E
104955,19da4d234f8ab01975d9d770564b826b_6085d8ec-973b...,D:\Users\Ryan\AppData\Local\Packages\Microsoft...,1481,1.569720e+09,0F071BB9
...,...,...,...,...,...
183846,readme.md,D:\Users\Ryan\AppData\Roaming\discord\0.0.308\...,580,1.607130e+09,BDBDFA21
69087,messages.json,D:\Users\Ryan\AppData\Local\Google\Chrome\User...,179,1.539055e+09,6B6C874F
242527,Paragraph I.doc,F:Files\Old\Iowa State\Pre Spring 2012\High Sk...,20480,1.178289e+09,073FBC7F
9446,31bc330521af537b7302aeadabd3d5d044ec7f,C:\Users\Ryan\Desktop\Files\corpus\.git\object...,213,1.618119e+09,653D3D10


# Data Cleansing

### Filter dataframe down to duplicate CRCs only

In [102]:
%%time
df = df[df['crc']!='00000000']
grouped = df.groupby('crc')
#grouped.filter('')
grouped = grouped.count().sort_values('path', ascending = False)
grouped = grouped[grouped['path']>1]
duplicatedCRCs = set(grouped.index)
df = df[df['crc'].isin(duplicatedCRCs) ]
grouped

Wall time: 399 ms


,name,path,size,mtime
crc,,,,
F05129F1,1143,1143,1143,1143
57F5D79F,156,156,156,156
2622E720,133,133,133,133
14A285AC,128,128,128,128
518E617C,126,126,126,126
...,...,...,...,...
13F89049,2,2,2,2
F021BB50,2,2,2,2
1882CD33,2,2,2,2


### Use this to explore and add more exclusions
If a directory comes up with 12,000 files in it and it's some Chrome cache file, you can probably ignore add it to your excludeDirs list.
Same idea for file extensions.

In [136]:
#Directory Exclusions
additionalExclusions =  {
    'Microsoft.XboxGamingOverlay_8wekyb3d8bbwe',
    'Dolphin',
    'cache',
    'Cache',
    'cache2',
    'Code Cache',
    'Vita',
    'TheDebianProject.DebianGNULinux_76v4gfsz19hv4',
    'RCTCache',
    'NVIDIA',
    '__pycache__',
    'site-packages',
    'TurboTax',    
    'Xbox',
    'CacheStorage',
    'CachedData',
    'CINEBENCH R15.038_RC184115',
    'solr',
    'dfl-venv',
    'MovieMaker',
}
excludeDirs.update(additionalExclusions)
print(f'before {len(df)=}')
df = df[[not excludeDirs.intersection(set(x.parts)) for x in df['path']]]
print(f'after {len(df)=}')

before len(df)=39793
after len(df)=39537


In [104]:
%%time

#let's try to find the most common folders files are in
df['parent'] = [x.parent for x in df['path']]
grouped = df.groupby('parent')
grouped.count().sort_values('name', ascending = False).head(10)

Wall time: 898 ms


,name,path,size,mtime,crc
parent,,,,,
C:\Users\Ryan\Desktop\Files\photos\Old Photos\Dump,1002,1002,1002,1002,1002
F:Files\photos\Old Photos\Dump,1002,1002,1002,1002,1002
G:photos\Old Photos\Dump,1002,1002,1002,1002,1002
G:projects\processing-3.0.1\modes\java\reference,639,639,639,639,639
C:\Users\Ryan\Desktop\Files\code\processing-3.0.1\modes\java\reference,639,639,639,639,639
F:Files\code\processing-3.0.1\modes\java\reference,639,639,639,639,639
G:code\processing-3.0.1\modes\java\reference,639,639,639,639,639
C:\Users\Ryan\Desktop\Files\code\Rpi\camera,591,591,591,591,591
G:photos\Old Photos\recovered pictures,356,356,356,356,356


In [105]:
#File Extension Exclusions
additionalExclusions =  {
    '.pyi',
    '.pde',
    '.html',
    None,
    '.xml',
    '.dll',
    '',
}
excludeExtensions.update(additionalExclusions)
print(f'before {len(df)=}')
df = df[[not excludeExtensions.intersection(set(x.suffixes)) for x in df['path']]]
print(f'after {len(df)=}')

before len(df)=51568
after len(df)=39793


In [107]:
#let's try to find the most common folders stuff is in
#df = 
df.loc[:,('suffix')] = [x.suffix for x in df['path']]
grouped = df.groupby('suffix')
grouped.count().sort_values('path', ascending = False).head(10)

,name,path,size,mtime,crc,parent
suffix,,,,,,
.jpg,9448,9448,9448,9448,9448,9448
.png,3702,3702,3702,3702,3702,3702
,3556,3556,3556,3556,3556,3556
.mkv,2801,2801,2801,2801,2801,2801
.py,1631,1631,1631,1631,1631,1631
.class,1476,1476,1476,1476,1476,1476
.mp3,1299,1299,1299,1299,1299,1299
.frm,1028,1028,1028,1028,1028,1028
.java,794,794,794,794,794,794


In [142]:
#build a django/flask front end
#have an html dataframe table with selection checkboxes
    #have the ability to select all, or just buttons to skip, ignore, remove from future compares, or delete non c drive dupes
        #buttons to exclude part of a given folder, a file extension
    #have the ability to move all contents from a folder with deletions into the c drive equivalent folder
        #this can only happen after duplication on the c drive is resolved
    #deleter moves to a recycle bin that can be restored from

#group by highest level parent and recursively build a tree
    #have some kind of a winDirStat visualization
    
dirs = [Path(x).as_posix() for x in dedupeDirs]
primary = dirs[0]
secondaries = dirs[1:]

try:
    del df['size']
    del df['mtime']
except:
    pass

cDrive    = df[[x.as_posix().startswith(posixDedupeDirs[0]) for x in df['path']]]
nonCDrive = df[[not x.as_posix().startswith(posixDedupeDirs[0]) for x in df['path']]]

merged = cDrive.merge(nonCDrive, on = ['crc'], suffixes = ('_c', '_dupe'))#, indicator = True)

mergedFolderGrouping = merged.groupby('parent_c')

mergedFolderGrouping = list(mergedFolderGrouping)
shuffle(mergedFolderGrouping)

for parent, group in mergedFolderGrouping:
    print(f'duplicates in folder {parent}')
    del group['parent_dupe']
    del group['name_dupe']
    del group['crc']
    del group['parent_c']
    del group['suffix_c']
    del group['suffix_dupe']
    group.rename(columns = {'name_c':'primary_name', 'path_c':'primary_path', 'path_dupe':'duplicate_file'}, inplace = True)
    group.set_index('primary_path', inplace = True)
    display(group)
    break
    clear_output()


duplicates in folder C:\Users\Ryan\Desktop\Files\code\processing-3.0.1\modes\java\bin\antlr


,primary_name,duplicate_file
primary_path,,
C:\Users\Ryan\Desktop\Files\code\processing-3.0.1\modes\java\bin\antlr\ExtendedCommonASTWithHiddenTokens.class,ExtendedCommonASTWithHiddenTokens.class,F:Files\code\processing-3.0.1\modes\java\bin\a...
C:\Users\Ryan\Desktop\Files\code\processing-3.0.1\modes\java\bin\antlr\ExtendedCommonASTWithHiddenTokens.class,ExtendedCommonASTWithHiddenTokens.class,G:code\processing-3.0.1\modes\java\bin\antlr\E...
C:\Users\Ryan\Desktop\Files\code\processing-3.0.1\modes\java\bin\antlr\ExtendedCommonASTWithHiddenTokens.class,ExtendedCommonASTWithHiddenTokens.class,G:projects\processing-3.0.1\modes\java\bin\ant...
C:\Users\Ryan\Desktop\Files\code\processing-3.0.1\modes\java\bin\antlr\TokenStreamCopyingHiddenTokenFilter.class,TokenStreamCopyingHiddenTokenFilter.class,G:code\processing-3.0.1\modes\java\bin\antlr\T...
C:\Users\Ryan\Desktop\Files\code\processing-3.0.1\modes\java\bin\antlr\TokenStreamCopyingHiddenTokenFilter.class,TokenStreamCopyingHiddenTokenFilter.class,G:projects\processing-3.0.1\modes\java\bin\ant...
C:\Users\Ryan\Desktop\Files\code\processing-3.0.1\modes\java\bin\antlr\TokenStreamCopyingHiddenTokenFilter.class,TokenStreamCopyingHiddenTokenFilter.class,F:Files\code\processing-3.0.1\modes\java\bin\a...


### Prioritize
  - Assign a root tier to each file
  - Assign a priority based on path length to resolve disputes within root tier### Assign a priority based on path length to resolve disputes within root tier

In [ ]:
%%time

def getTier(file):
    file = file.as_posix()
    return int([rootTier for rootTier, posix_path in dedupeTiers if file.startswith(posix_path)][0])

dedupeTiers = [(rootTier,Path(directory).as_posix()) for rootTier,directory in enumerate(dedupeDirs)]
df['rootTier'] = df['path'].apply(getTier)

#prioritize shortest len paths
df['pathLen'] = df['path'].map(Path.as_posix).str.len()
df.sort_values(by= ['crc','name','rootTier', 'pathLen'], inplace = True)
df['priority'] = [cnt for cnt, x in enumerate(df.index)]
del df['pathLen']
df

In [84]:
x = Path('C:/Users/Ryan/Desktop/')
dir('')

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'removeprefix',
 'removesuffix',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',


# HERE BE DRAGONS

### TODO:
I can't go duplicate by duplicate. There's too much and I risk breaking files by deleting common needed components.  I need to know if an entire subdir is an exact match, and match high up the trees.  Basically match on folder name, folder contents, number of stuff in folder
 - maybe do something like find a folder with name Mom Pictures
   - identify exact folder name matches
   - fina all matches
   - crc different folder instances with all subdirs
   - copy from all other drives to primary if crc does not exist
       - later add step to  delete duplicated non primary copies
   - do a folder by folder approach


### Only run on exact duplicates first

In [53]:
#TODO: make the index and crc one pool process?

In [ ]:
grouped = list(df.groupby('parent'))#.sort_values(len(path.parts))
display(grouped[0][1])

#for group in grouped, start deleting stuff
#for _, dupeGroup in grouped:
#    dupeGroup.sort_values('rootTier', inplace = True)
#    display(dupeGroup)
#    input()
#    clear_output()
    
    
    
    #print(type(dupeGroup))
#    print(dupeGroup)
    #dupeGroup = 
    #print(type(dupeGroup))
    #print([x for x in dupeGroup])
    #clear_output()
    #display(HTML(dupeGroup.to_html()))
    
#    input('press enter to continue')

    #print(dupeGroup)


In [ ]:
#shows how many duplicate groups ("path_Count") we have by number of duplicate files
print(len(grouped))
dupeGroupSizeCount = grouped.copy().add_suffix('_Count').reset_index()
dupeGroupSizeCount = dupeGroupSizeCount.groupby('name_Count').count()[['path_Count']]
dupeGroupSizeCount

### let's do some analysis by file suffixes

In [ ]:
suffixesCounter = Counter()

for k,dupeList in dupeDict.items(): 
    for file in dupeList:
        #for suffix in file['pathObj'].suffixes:
        suffixesCounter[file['pathObj'].suffix] += 1
suffixesCounter = [(k,v) for k,v in suffixesCounter.items()]
suffixesCounter.sort(key = lambda k: k[1], reverse=True)

#top 20 duplicated 
pprint(suffixesCounter[:20])

### start by figuring out duplication categories
 - same name, same size, same mtime, same hash different dedupeDirs
     - can safely delete one of them based on priority
 - same name, same size, different mtime
     - hash and branch logic if different
 - same name, different size 
     - send to manual review queue, potentially prefer the newer one

For my deletion logic I need the dry run I need to output a file that would say here's the original path and here's the moved to path which is basically the recycle bin and then I need to be able to restore files to and from that recycle bin before permanent deletion

Make sure this code to restore stuff from the recycle bin as well tested so that I don't break any processes I can't remember

All unique files on my non-primary drive need to be pulled over and brought into the formal file structure

Look into whatever line is tech tips use that was basically the r seeing log rotate thing where I can automate my backups I need to have snapshots as well as like I don't schedule to

# TODO:
group by a combination of name, size, crc.
if the crc checks match, analyze 
group by analysis has an order by clause, keep the top x and have a function for delete with dry run as the base

### Let's get deleting files
Work out the priority of each file to conslidate everything on one disk first.

Dry run is enabled unless you disable it.

In [ ]:
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}" 
        num /= 1024.0
    return "{num:.1f}Yi{suffix}"


In [ ]:
from IPython.display import display, HTML

# Assuming that dataframes df1 and df2 are already defined:

#display(df)
display(df.to_html())


dryRun = True

## TODO:
 - compare by file attributes, basically just size and name
     -  then by hash if same name/size
 - if name/size disagree, put in a conflict csv to review later


In [ ]:
#%time
#my group by root tier logic seems to be working 
grouped = df.groupby('crc', sort = False)

#for cnt, (crc, group) in enumerate(grouped):
#    if len(group) == 1 or crc == '00000000':
#        continue
    #if group['rootTier'].min() == group['rootTier'].max():
    #    continue
    #clear_output()
    #display(group[group['rootTier'] == group['rootTier'].max()])
    #display(group)
    #time.sleep(4)



Bold 	**bold text**
*italicized text*

Blockquote 	> blockquote
Ordered List 	
1. First item
2. Second item
3. Third item
Unordered List 	
- First item
- Second item
- Third item
Code 	`code`
Horizontal Rule 	---
Link 	
[title](https://www.example.com)
Image 	
![alt text](image.jpg)

### Results 
| Stretch/Untouched | ProbDistribution | Accuracy |
| :- | -: | :-: |
| Stretched | Gaussian | .843

 ```
{
  "firstName": "John",
  "lastName": "Smith",
  "age": 25
}
``` 
footnote 	Here's a sentence with a footnote. [^1]

[^1]: This is the footnote.
Heading ID 
### My Great Heading {#custom-id}
Definition List 	
term
: definition

Task List 	
- [x] Write the press release
- [ ] Update the website

